<h1>Instalando Dependencias Necesarias</h1>

In [17]:
%%capture
%pip install -U sentence-transformers pandas

In [18]:
import pandas as pd

<h2>Entendiendo el Dataset</h2>

In [13]:
df = pd.read_csv('Pelis1000.csv')

In [15]:
# Ver Primeros elementos del dataframe
df.head()

,Unnamed: 0,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info
0,0,1. The Shawshank Redemption (1994),R,142 min,Drama,9.3,80.0,Two imprisoned men bond over a number of years...,"Director: Frank Darabont | Stars: Tim Robbins,...","Votes: 2,295,987 | Gross: $28.34M"
1,1,2. The Godfather (1972),R,175 min,"Crime, Drama",9.2,100.0,The aging patriarch of an organized crime dyna...,Director: Francis Ford Coppola | Stars: Marlon...,"Votes: 1,584,782 | Gross: $134.97M"
2,2,3. The Dark Knight (2008),PG-13,152 min,"Action, Crime, Drama",9.0,84.0,When the menace known as the Joker wreaks havo...,Director: Christopher Nolan | Stars: Christian...,"Votes: 2,260,649 | Gross: $534.86M"
3,3,4. The Godfather: Part II (1974),R,202 min,"Crime, Drama",9.0,90.0,The early life and career of Vito Corleone in ...,Director: Francis Ford Coppola | Stars: Al Pac...,"Votes: 1,107,253 | Gross: $57.30M"
4,4,5. The Lord of the Rings: The Return of the Ki...,PG-13,201 min,"Action, Adventure, Drama",8.9,94.0,Gandalf and Aragorn lead the World of Men agai...,"Director: Peter Jackson | Stars: Elijah Wood, ...","Votes: 1,614,369 | Gross: $377.85M"


In [41]:
# Muestre la información de los datos de cada Columna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   1000 non-null   int64  
 1   Title        1000 non-null   object 
 2   Certificate  973 non-null    object 
 3   Duration     1000 non-null   object 
 4   Genre        1000 non-null   object 
 5   Rate         1000 non-null   float64
 6   Metascore    712 non-null    float64
 7   Description  1000 non-null   object 
 8   Cast         1000 non-null   object 
 9   Info         1000 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 78.3+ KB


In [109]:
# Muestre la información de las columnas categoricas
df.describe()

,Unnamed: 0,Rate,Metascore
count,1000.000000,1000.000000,712.000000
mean,499.500000,8.097500,81.001404
std,288.819436,0.169566,9.811801
min,0.000000,8.000000,61.000000
25%,249.750000,8.000000,73.000000
50%,499.500000,8.000000,82.000000
75%,749.250000,8.100000,88.250000
max,999.000000,9.300000,100.000000


In [111]:
# Muestre el numero de filas x columnas
df.shape

(1000, 11)

In [113]:
# Muestre los tipos de datos que existen en las columnas
df.dtypes

Unnamed: 0       int64
Title           object
Certificate     object
Duration        object
Genre           object
Rate           float64
Metascore      float64
Description     object
Cast            object
Info            object
embeddings      object
dtype: object

<h3>
Usando Sentence Transformer para Crear Embeddings
</h3>

In [50]:
# Instalcion de librerias
!pip install sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer, util

C:\Users\ELVIN TORRES\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [21]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [34]:
embeddings = model.encode(df['Description'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [25]:
# Visualizar los Primeros elementos del dataframe
print(df.head())

   Unnamed: 0                                              Title Certificate  \
0           0                 1. The Shawshank Redemption (1994)           R   
1           1                            2. The Godfather (1972)           R   
2           2                          3. The Dark Knight (2008)       PG-13   
3           3                   4. The Godfather: Part II (1974)           R   
4           4  5. The Lord of the Rings: The Return of the Ki...       PG-13   

  Duration                     Genre  Rate  Metascore  \
0  142 min                     Drama   9.3       80.0   
1  175 min              Crime, Drama   9.2      100.0   
2  152 min      Action, Crime, Drama   9.0       84.0   
3  202 min              Crime, Drama   9.0       90.0   
4  201 min  Action, Adventure, Drama   8.9       94.0   

                                         Description  \
0  Two imprisoned men bond over a number of years...   
1  The aging patriarch of an organized crime dyna...   
2  When 

In [29]:
# imprima los nombres de las columnas
print(df.columns)

Index(['Unnamed: 0', 'Title', 'Certificate', 'Duration', 'Genre', 'Rate',
       'Metascore', 'Description', 'Cast', 'Info'],
      dtype='object')


In [38]:
df['embeddings'] = embeddings.tolist()

<h1>
Calculando la Similitud usando la Métrica de Similitud por Coseno
</h1>

In [42]:
def compute_similarity(example, query_embedding):
    embedding = example['embeddings'] 
    similarity = util.cos_sim(embedding, query_embedding).item()
    return similarity  

<h1>Ejecuntando la búsqueda</h1>

In [46]:
query_embedding = model.encode(['matrix'])[0]
df['similarity'] = df.apply(lambda x: compute_similarity(x, query_embedding), axis=1)
df = df.sort_values(by='similarity', ascending=False)

df.head()['Title']

15              16. The Matrix (1999)
691    394. Ghost in the Shell (1995)
591    394. Ghost in the Shell (1995)
491    394. Ghost in the Shell (1995)
991    394. Ghost in the Shell (1995)
Name: Title, dtype: object

<h1>Crear una función que se cree una nueva columna que va a tener información relevante para los embeddings, .tal vez es importante tener el valor ganado de la película, o el nombre del director</h1>

In [160]:
print(df.columns)

Index(['Unnamed: 0', 'Title', 'Certificate', 'Duration', 'Genre', 'Rate',
       'Metascore', 'Description', 'Cast', 'Info', 'embeddings'],
      dtype='object')


In [168]:
df_clasificado = df.sort_values(by='Info', ascending=False) 

In [164]:
print(df_clasificado.head())

     Unnamed: 0                                  Title Certificate Duration  \
16           16                  17. Goodfellas (1990)           R  146 min   
237         237          240. Kill Bill: Vol. 1 (2003)           R  111 min   
65           65                      66. WALL·E (2008)           G   98 min   
43           43  44. Terminator 2: Judgment Day (1991)           R  137 min   
189         189              190. The Gold Rush (1925)      Passed   95 min   

                            Genre  Rate  Metascore  \
16        Biography, Crime, Drama   8.7       90.0   
237       Action, Crime, Thriller   8.1       69.0   
65   Animation, Adventure, Family   8.4       95.0   
43                 Action, Sci-Fi   8.5       75.0   
189      Adventure, Comedy, Drama   8.2        NaN   

                                           Description  \
16   The story of Henry Hill and his life in the mo...   
237  After awakening from a four-year coma, a forme...   
65   In the distant future

In [ ]:
Director: Andrew Stanton | Stars: Ben Burtt, valor aproximado de la pelicula : $223.81M   